In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
sys.tracebacklimit = 0

import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

In [3]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *
from solution.policy_net import *

In [4]:
from core.agent import *
from core.world import * 
from core.render import * 
from core.skill import * 
from core.models import *
from core.message import *

In [5]:
DEVICE = "cuda"

In [6]:
from sar.sar_agent import *
from sar.sar_world import *
from sar.sar_env_params import *
from sar.sar_traits_sampler import *

belief_initializer = SARBeliefInitializer(BELIEF_DIMS)
trait_sampler = SARTraitSampler()

def initialize_swarm(world : BaseWorld):
    swarm = trait_sampler.generate(SWARM_SIZE, DEVICE)
    for agent in swarm:
        agent.set_utility(SARUtilityFunction())
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    swarm = belief_initializer.initialize_beliefs(swarm)

In [7]:

from sar.urban_gen import * 
from sar.victims import * 
from sar.sar_comm import * 

terrain_generator = UrbanTerrainMapGenerator(padding = MAX_VISIBILITY)
victim_generator = VictimGenerator(padding = MAX_VISIBILITY)
def initialize_terrain(world : BaseWorld):
    terrain_map, population_map = terrain_generator.generate(world._dims)
    map_collection : BaseMapCollection = BaseMapCollection()
    map_collection.add_map("Terrain", terrain_map)
    map_collection.add_map("Population", population_map)

    victim_generator.set_density_map(population_map)
    victim_map = victim_generator.generate(world._dims)

    map_collection.add_map("Victims", victim_map)
    return map_collection


In [8]:
from sar.energy import EnergyModel
from sar.victims import VictimModel
from solution.sar_action_interpreter import *
from solution.encoder_net import *
from solution.decoder_net import *
from models.complex_model import * 

world = SARWorld(dims = WORLD_DIMS,
              swarm_initializer= initialize_swarm,
              generation_pipeline=initialize_terrain
              )
world.add_model("energy_model", EnergyModel())
world.add_model("victim_model", VictimModel())
world.reset()

In [ ]:
grid_size = 2 * 3 + 1
policy_net= PolicyNet(1, grid_size, 4)
target_net=  PolicyNet(1, grid_size, 4)
encoder_model = Encoder()
decoder_model = Decoder()

complex_model = ComplexModel(
    policy_net= policy_net, 
    encoder_net = encoder_model, 
    decoder_net = decoder_model
)

comms_protocol = SARCommunicationProtocol(encoder_model, decoder_model)
action_interpreter = SARActionInterpreter(BELIEF_DIMS)

custom_gym : CustomGymEnviornment = CustomGymEnviornment(world, action_interpreter, comms_protocol)

complex_model.to(DEVICE)
custom_gym.to(DEVICE)

In [10]:
custom_gym.reset(42)

({1: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 0., 0., 1., 1.],
          [0., 0., 1., 0., 0.],
          [1., 0., 0., 0., 1.],
          [0., 0., 1., 0., 0.],
          [1., 0., 0., 0., 0.]]),
   'State': tensor([90.3885,  0.0000,  0.0000], device='cuda:0')},
  2: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 1., 1., 0.],
          [1., 1., 1., 0., 1.],
          [0., 1., 0., 0., 1.],
          [1., 1., 1., 0., 1.],
          [0., 0., 0., 0., 0.]]),
   'State': tensor([107.1394,   1.0000,   1.0000], device='cuda:0')},
  3: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 0., 1.],
          [0., 0., 0.],
          [0., 1., 1.]]),
   'State': tensor([99.2335,  1.0000,  1.0000], device='cuda:0')},
  4: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 0., 1., 0.],
          [1., 1., 1., 0., 1.],
          [1., 0., 0., 1., 1.],
          

### Testing

In [11]:
parallel_api_test(custom_gym, num_cycles=1_000)
custom_gym.reset()

Passed Parallel API test


({1: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 0., 0., 1., 1.],
          [0., 0., 1., 0., 0.],
          [1., 0., 0., 0., 1.],
          [0., 0., 1., 0., 0.],
          [1., 0., 0., 0., 0.]]),
   'State': tensor([90.3885,  0.0000,  0.0000], device='cuda:0')},
  2: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 1., 1., 0.],
          [1., 1., 1., 0., 1.],
          [0., 1., 0., 0., 1.],
          [1., 1., 1., 0., 1.],
          [0., 0., 0., 0., 0.]]),
   'State': tensor([107.1394,   1.0000,   1.0000], device='cuda:0')},
  3: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 0., 1.],
          [0., 0., 0.],
          [0., 1., 1.]]),
   'State': tensor([99.2335,  1.0000,  1.0000], device='cuda:0')},
  4: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 0., 1., 0.],
          [1., 1., 1., 0., 1.],
          [1., 0., 0., 1., 1.],
          

# Training

In [12]:

from models.base import * 
from models.idqn import * 
from solution.policy_net import PolicyNet
import matplotlib.pyplot as plt

In [13]:
model = IDQN(env = custom_gym,
             feature_extractor= feature_extractor,
             target_net= target_net,
             model= complex_model,
             batch_size=1024,
             device = DEVICE
             )

In [ ]:
rewards = train_loop(custom_gym, model, games=100, optimization_passes = 1, seed=42)

Training on thesis.


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]c:\Users\Joaquin\miniconda3\envs\thesis\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1, 1024])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Average loss 0.12871457014232873
Model has been saved.

Starting evaluation on thesis (num_games=1)


Training Progress:   1%|          | 1/100 [00:09<15:29,  9.38s/it]

Avg reward: 15.26  std: 6.71210846157897  coeff : 0.43984983365524055
Avg reward per agent, per game:  {1: 31.0, 2: 10.0, 3: 19.0, 4: 23.0, 5: 26.0, 6: 28.0, 7: 19.0, 8: 12.0, 9: 7.0, 10: 10.0, 11: 11.0, 12: 20.0, 13: 9.0, 14: 15.0, 15: 18.0, 16: 17.0, 17: 15.0, 18: 14.0, 19: 33.0, 20: 25.0, 21: 9.0, 22: 22.0, 23: 23.0, 24: 14.0, 25: 15.0, 26: 9.0, 27: 16.0, 28: 12.0, 29: 20.0, 30: 5.0, 31: 9.0, 32: 11.0, 33: 8.0, 34: 10.0, 35: 8.0, 36: 8.0, 37: 11.0, 38: 10.0, 39: 17.0, 40: 16.0, 41: 15.0, 42: 26.0, 43: 18.0, 44: 8.0, 45: 12.0, 46: 18.0, 47: 10.0, 48: 27.0, 49: 9.0, 50: 11.0, 51: 21.0, 52: 9.0, 53: 23.0, 54: 15.0, 55: 11.0, 56: 14.0, 57: 2.0, 58: 9.0, 59: 28.0, 60: 11.0, 61: 10.0, 62: 10.0, 63: 33.0, 64: 10.0, 65: 13.0, 66: 21.0, 67: 17.0, 68: 9.0, 69: 9.0, 70: 15.0, 71: 8.0, 72: 12.0, 73: 16.0, 74: 14.0, 75: 10.0, 76: 28.0, 77: 23.0, 78: 21.0, 79: 10.0, 80: 15.0, 81: 15.0, 82: 5.0, 83: 13.0, 84: 5.0, 85: 14.0, 86: 32.0, 87: 22.0, 88: 10.0, 89: 12.0, 90: 15.0, 91: 22.0, 92: 18.0, 93: 

...

Avg reward: 17.55  std: 8.854800957672621  coeff : 0.5045470631152491
Avg reward per agent, per game:  {1: 26.0, 2: 15.0, 3: 22.0, 4: 23.0, 5: 40.0, 6: 17.0, 7: 10.0, 8: 24.0, 9: 10.0, 10: 17.0, 11: 17.0, 12: 21.0, 13: 12.0, 14: 17.0, 15: 38.0, 16: 25.0, 17: 10.0, 18: 14.0, 19: 36.0, 20: 28.0, 21: 20.0, 22: 32.0, 23: 25.0, 24: 18.0, 25: 3.0, 26: 6.0, 27: 29.0, 28: 21.0, 29: 11.0, 30: 8.0, 31: 24.0, 32: 11.0, 33: 8.0, 34: 15.0, 35: 9.0, 36: 11.0, 37: 20.0, 38: 16.0, 39: 19.0, 40: 18.0, 41: 20.0, 42: 36.0, 43: 8.0, 44: 37.0, 45: 12.0, 46: 30.0, 47: 16.0, 48: 15.0, 49: 13.0, 50: 13.0, 51: 27.0, 52: 7.0, 53: 24.0, 54: 31.0, 55: 9.0, 56: 7.0, 57: 5.0, 58: 27.0, 59: 12.0, 60: 7.0, 61: 13.0, 62: 22.0, 63: 47.0, 64: 10.0, 65: 7.0, 66: 16.0, 67: 5.0, 68: 13.0, 69: 18.0, 70: 13.0, 71: 22.0, 72: 7.0, 73: 17.0, 74: 22.0, 75: 20.0, 76: 21.0, 77: 15.0, 78: 21.0, 79: 7.0, 80: 13.0, 81: 13.0, 82: 6.0, 83: 14.0, 84: 9.0, 85: 15.0, 86: 23.0, 87: 26.0, 88: 21.0, 89: 27.0, 90: 9.0, 91: 10.0, 92: 25.0, 93:

Training Progress:  52%|█████▏    | 52/100 [09:02<08:39, 10.82s/it]

Avg reward: 17.64  std: 8.856093947107833  coeff : 0.5020461421262944
Avg reward per agent, per game:  {1: 23.0, 2: 10.0, 3: 24.0, 4: 23.0, 5: 35.0, 6: 16.0, 7: 16.0, 8: 23.0, 9: 10.0, 10: 18.0, 11: 20.0, 12: 11.0, 13: 13.0, 14: 17.0, 15: 34.0, 16: 12.0, 17: 15.0, 18: 11.0, 19: 51.0, 20: 22.0, 21: 11.0, 22: 23.0, 23: 10.0, 24: 17.0, 25: 11.0, 26: 10.0, 27: 28.0, 28: 27.0, 29: 30.0, 30: 4.0, 31: 24.0, 32: 20.0, 33: 11.0, 34: 9.0, 35: 10.0, 36: 11.0, 37: 19.0, 38: 15.0, 39: 28.0, 40: 29.0, 41: 16.0, 42: 17.0, 43: 24.0, 44: 18.0, 45: 11.0, 46: 28.0, 47: 17.0, 48: 8.0, 49: 28.0, 50: 9.0, 51: 30.0, 52: 5.0, 53: 16.0, 54: 6.0, 55: 15.0, 56: 7.0, 57: 7.0, 58: 21.0, 59: 20.0, 60: 7.0, 61: 14.0, 62: 18.0, 63: 35.0, 64: 9.0, 65: 6.0, 66: 21.0, 67: 12.0, 68: 9.0, 69: 21.0, 70: 17.0, 71: 25.0, 72: 8.0, 73: 19.0, 74: 17.0, 75: 28.0, 76: 18.0, 77: 7.0, 78: 31.0, 79: 3.0, 80: 15.0, 81: 25.0, 82: 5.0, 83: 30.0, 84: 14.0, 85: 20.0, 86: 27.0, 87: 30.0, 88: 16.0, 89: 26.0, 90: 15.0, 91: 4.0, 92: 34.0, 93

Training Progress:  53%|█████▎    | 53/100 [09:14<08:35, 10.97s/it]

Avg reward: 17.26  std: 9.118793779881196  coeff : 0.528319454222549
Avg reward per agent, per game:  {1: 18.0, 2: 3.0, 3: 19.0, 4: 24.0, 5: 47.0, 6: 11.0, 7: 19.0, 8: 27.0, 9: 20.0, 10: 8.0, 11: 14.0, 12: 14.0, 13: 26.0, 14: 15.0, 15: 25.0, 16: 11.0, 17: 20.0, 18: 9.0, 19: 51.0, 20: 25.0, 21: 13.0, 22: 19.0, 23: 36.0, 24: 21.0, 25: 12.0, 26: 10.0, 27: 21.0, 28: 10.0, 29: 24.0, 30: 2.0, 31: 10.0, 32: 15.0, 33: 22.0, 34: 10.0, 35: 7.0, 36: 9.0, 37: 20.0, 38: 18.0, 39: 43.0, 40: 33.0, 41: 21.0, 42: 34.0, 43: 10.0, 44: 14.0, 45: 14.0, 46: 10.0, 47: 20.0, 48: 20.0, 49: 20.0, 50: 15.0, 51: 27.0, 52: 6.0, 53: 16.0, 54: 5.0, 55: 10.0, 56: 6.0, 57: 8.0, 58: 20.0, 59: 9.0, 60: 12.0, 61: 7.0, 62: 22.0, 63: 27.0, 64: 13.0, 65: 14.0, 66: 17.0, 67: 18.0, 68: 8.0, 69: 19.0, 70: 18.0, 71: 29.0, 72: 13.0, 73: 36.0, 74: 17.0, 75: 9.0, 76: 24.0, 77: 11.0, 78: 21.0, 79: 13.0, 80: 16.0, 81: 25.0, 82: 8.0, 83: 15.0, 84: 9.0, 85: 11.0, 86: 19.0, 87: 16.0, 88: 14.0, 89: 29.0, 90: 29.0, 91: 7.0, 92: 14.0, 93:

In [ ]:
plt.plot(rewards)

# Optimizations

In [ ]:
import cProfile

In [ ]:
def stress_test():
    train_loop(custom_gym, model, games=1, optimization_passes = 1, seed=42)

In [ ]:

cProfile.run('stress_test()', sort = 'time')

# Visualization

In [ ]:
from models.base import RenderWrapper

renderer = RenderWrapper(model=model)

In [ ]:
renderer.render()